# PYTORCH MODULE
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- [How to Use PyTorch](https://greeksharifa.github.io/pytorch/2018/11/10/pytorch-usage-03-How-to-Use-PyTorch/#pytorch-model)

## torch.nn.Module

In [114]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module): 
    def __init__(self):                     
        # 모델에 사용될 module을 정의
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)
        self.flat = nn.Flatten(20)
    
    def forward(self, x) :                  
        # train 중 forward 계산할 때 모델에서 작동하는 계산을 정의 (__init__에서 정의한 모듈 그래도 사용), 
        # backward계산은 나중에 backward() 호출하면 계산해준다.
        x = F.relu(self.conv1)
        x = F.relu(self.conv2(x))
        return F.relu(self.flat(x))

model = Model()
model

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
)

In [129]:
# add_module(name, module) : Adds a child module to the current module.
dense_layer = nn.Linear(20, 32)
model.add_module('flatten', dense_layer)
model

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)

In [116]:
# apply(fn) : Applies fn recursively to every submodule (as returned by .children()) as well as self.
# nitializing the parameters of a model 할 때 많이 사용함

@torch.no_grad()
def init_weight(m):
    print(m)
    if type(m) == nn.Linear :
        m.weight.fill_(1.0) # weight를 1.0으로 초기화
        print(m.weight)
        print("><><><><><")

net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weight)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
><><><><><
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
><><><><><
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [117]:
# children() : Returns an iterator over immediate children modules.

for idx, layer in enumerate(model.children()):
    print(idx, "->", layer)

0 -> Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
1 -> Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
2 -> Flatten(start_dim=20, end_dim=-1)
3 -> Linear(in_features=20, out_features=32, bias=True)


In [118]:
# modules() : Returns an iterator over all modules in the network.
for idx, layer in enumerate(model.modules()):
    print(idx, "->", layer)

0 -> Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)
1 -> Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
2 -> Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
3 -> Flatten(start_dim=20, end_dim=-1)
4 -> Linear(in_features=20, out_features=32, bias=True)


In [119]:
# named_buffers() : 이름까지 반환 
# Returns an iterator over immediate children modules, yielding both the name of the module as well as the module itself.

for name, layer in model.named_children():
    print(name, ":", layer)

conv1 : Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
conv2 : Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
flat : Flatten(start_dim=20, end_dim=-1)
flatten : Linear(in_features=20, out_features=32, bias=True)


In [120]:
# parameters(recurse=True) : Returns an iterator over module parameters.

for param in model.parameters():
    print(type(param), param.size())

<class 'torch.nn.parameter.Parameter'> torch.Size([20, 1, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([20])
<class 'torch.nn.parameter.Parameter'> torch.Size([20, 20, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([20])
<class 'torch.nn.parameter.Parameter'> torch.Size([32, 20])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])


In [121]:
# state_dict(destination=None, prefix='', keep_vars=False) 모델 저장
# Returns a dictionary containing a whole state of the module.
model.state_dict()
model.state_dict().keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'flatten.weight', 'flatten.bias'])

In [122]:
# load_state_dict(state_dict, strict=True) 
# Copies parameters and buffers from state_dict into this module and its descendants.
# If strict is True, then the keys of state_dict must exactly match the keys returned by this module’s state_dict() function.


In [123]:
# cuda(device=None) Moves all model parameters and buffers to the GPU.
# it should be called before constructing optimizer if the module will live on GPU while being optimized.
model.cuda()

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)

In [124]:
# eval() : Sets the module in evaluation mode.
# This is equivalent with self.train(False).

model.eval()

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)

In [125]:
# train(mode=True) : Sets the module in training mode.
model.train()

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)

In [126]:
# to(*args, **kwargs) : Moves and/or casts the parameters and buffers.

# dtype (torch.dtype) : the desired floating point or complex dtype of the parameters and buffers in this module
model.to(torch.double) 

# device (torch.device) : the desired device of the parameters and buffers in this module
gpu1 = torch.device("cuda")
model.to(gpu1, dtype=torch.half, non_blocking=True) 

cpu = torch.device("cpu")
model.to(cpu)


Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)

In [135]:
# requires_grad_(requires_grad=True) : Change if autograd should record operations on parameters in this module.
model.requires_grad_

<bound method Module.requires_grad_ of Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  (flat): Flatten(start_dim=20, end_dim=-1)
  (flatten): Linear(in_features=20, out_features=32, bias=True)
)>

In [134]:
linear = nn.Linear(2, 2)
linear.weight

Parameter containing:
tensor([[-0.5519, -0.2719],
        [ 0.2158,  0.1707]], requires_grad=True)